# Build a Sentiment Analysis Model with Scikit Learn

With Amazon SageMaker

source: https://www.twilio.com/blog/2017/12/sentiment-analysis-scikit-learn.html

First, import the necessary libraries:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
import random
import boto3
import warnings

warnings.filterwarnings(action='ignore', category=Warning) 

Declare and initialize the s3 resource. Define constant variables that hold the value of file paths. Replace ```your-bucket-name``` with the name of the bucket you created for this workshop.

In [ ]:
s3 = boto3.resource('s3')
BUCKET_NAME = '<your-bucket-name>'
MODEL_FILE_KEY = 'ServerlessAIWorkshop/SentimentAnalysis/sentiment_analysis_tweet.pkl'
X_TEST_FILE_KEY = 'ServerlessAIWorkshop/SentimentAnalysis/X_test.data'
y_TEST_FILE_KEY = 'ServerlessAIWorkshop/SentimentAnalysis/y_test.data'

Declare array objects to store dataset. Data and label will be stored separately.

In [ ]:
data = []
data_labels = [] #label will be either 'pos' or 'neg'

Open and read each dataset and append to the array objects created above.

In [ ]:
with open("./pos_tweets.txt") as f:
    for i in f: 
        data.append(i) 
        data_labels.append('pos')

with open("./neg_tweets.txt") as f:
    for i in f: 
        data.append(i)
        data_labels.append('neg')

Vectorize the tweets content and convert to a two-dimensional array of word counts. This conversion to array is necessary to split operation.

In [ ]:
vectorizer = CountVectorizer(
    analyzer = 'word', # exclude common words such as “the” or “and”
    lowercase = False,
)
features = vectorizer.fit_transform(
    data
)
features_nd = features.toarray() 

Split the training data to get an evaluation set.
* X = dataset
* y = label
* train_ = training dataset
* test_ = validation dataset

In [ ]:
train_size = 0.8
seed = 7
X_train, X_test, y_train, y_test  = train_test_split(
        features_nd, 
        data_labels,
        train_size=train_size, 
        random_state=seed)

Build a classifier using sckit learn's logistic regression.

In [ ]:
log_model = LogisticRegression() # using LogisticRegression class from Sciki-learn
log_model = log_model.fit(X=X_train, y=y_train) # train the model using the training dataset

Run inference using the test/validation dataset. Sklearn.metrics.accuracy_score calculates what percentage of tweets are classified correctly.

In [ ]:
y_pred = log_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

Use scikit learns joblib module to first write the trained model and test dataset that is in memory into files (.pkl and .csv) on the notebook instance. Upload those artifacts to your S3 bucket.

In [ ]:
# save the trained model (in the memory) to a pickle file
joblib.dump(log_model, 'sentiment_analysis_tweet.pkl') 
joblib.dump(X_test, 'X_test.csv')
joblib.dump(y_test, 'y_test.csv')

# upload the trailed model/pickle file as well as test dataset to S3 bucket
s3.meta.client.upload_file('sentiment_analysis_tweet.pkl', BUCKET_NAME, MODEL_FILE_KEY)
s3.meta.client.upload_file('X_test.csv', BUCKET_NAME, X_TEST_FILE_KEY)
s3.meta.client.upload_file('y_test.csv', BUCKET_NAME, y_TEST_FILE_KEY)